In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import numpy as np
import time
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
from tqdm import tqdm
from torch.autograd import Variable
from Logger import Logger
from utils import *
from DataLoader import DataLoader
from Discriminator import Discriminator
from Generator import Generator
import cv2

C:\Users\pasan\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


torch.Size([3, 192, 256])
torch.Size([1, 192, 256])


In [2]:
logger = Logger(r'C:\Users\pasan\Documents\Notebooks\Saliency\logger.txt')
batch_size = 20
lr = 0.0003

In [3]:
discriminator = Discriminator()
generator = Generator()
if torch.cuda.is_available():
    discriminator.cuda()
    generator.cuda()
loss_function = nn.BCELoss()

Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) ReLU(inplace) Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) ReLU(inplace) MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False) Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) ReLU(inplace) Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) ReLU(inplace) MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False) Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) ReLU(inplace) Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) ReLU(inplace) MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False) Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) ReLU(inplace) Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) ReLU(inplace) Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) ReLU(inplace) MaxPool2d(kernel_size=2, stride=2, padding=

In [4]:
print(discriminator.parameters(), generator.parameters())
d_optim = torch.optim.Adagrad(discriminator.parameters(), lr = lr)
g_optim = torch.optim.Adagrad(generator.parameters(), lr = lr)

<generator object Module.parameters at 0x0000026915DB0830> <generator object Module.parameters at 0x0000026915DB0A98>


In [5]:
num_epoch = 3 #125 for actual training gpu bound
dataloader = DataLoader(batch_size)
num_batch = dataloader.num_batches
num_batch = int(num_batch)
print(num_batch)

500


In [6]:
def to_variable(x, requires_grad = True):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x, requires_grad) 

In [7]:
counter = 0
g_loss = 0
fake_score = 0
start_time = time.time()
DIR_TO_SAVE = r"C:\Users\pasan\Documents\Notebooks\Saliency\SavedStates\generator_output"
if not os.path.exists(DIR_TO_SAVE):
    os.makedirs(DIR_TO_SAVE)
validation_sample = cv2.imread(r"C:\Users\pasan\Documents\Notebooks\Saliency\ResizedImages\images256x192_val\COCO_val2014_000000143859.png")
validation_sample = cv2.cvtColor(validation_sample, cv2.COLOR_BGR2RGB)
validation_sample = Image.fromarray(validation_sample)

for current_epoch in tqdm (range(1, num_epoch+1)):
    n_updates = 1
    d_cost_avg = 0
    g_cost_avg = 0
    
    for i in range(3):
        
        (batch_img, batch_map) = dataloader.get_batch()
        batch_img = to_variable(batch_img, requires_grad=True)
        batch_map = to_variable(batch_map, requires_grad=False)
        real_labels = to_variable(torch.FloatTensor(np.ones(batch_size, dtype = float)), requires_grad = False)
        fake_labels = to_variable(torch.FloatTensor(np.zeros(batch_size, dtype = float)), requires_grad = False)
        
        if n_updates % 2 == 1:
            
            print("Training Discriminator...")
            d_optim.zero_grad()
            inp_d = torch.cat((batch_img, batch_map), 1)
            if i == 1 :
                print('INP_D', inp_d)
            outputs = discriminator(inp_d).squeeze()
            d_real_loss = loss_function(outputs, real_labels)
            print('D_real_loss = ', d_real_loss.data[0])
            print('OUTPUTS', outputs)
            real_score = outputs.data.mean()
            
            d_loss = torch.sum(torch.log(outputs))
            print('D_LOSS', d_loss)
            d_cost_avg += d_loss.data[0]
            
            d_loss.backward()
            d_loss.register_hook(print)
            d_optim.step()
            
#             info = {'d_loss' : d_loss.data[0],
#                  'real_score_mean' : real_score,}
           
#             for tag, value in info.items():
#                 logger.scalar_summary(tag, value, counter)
        else :
            print('Training Generator...')
            g_optim.zero_grad()
            fake_map = generator(batch_img)
            inp_d = torch.cat((batch_img,fake_map),1)
            outputs = discriminator(inp_d)
            fake_score = outputs.data.mean()
            
            g_gen_loss = loss_function(fake_map, batch_map)
            print('g_gen_loss', g_gen_loss)
            g_dis_loss = -torch.log(outputs)
            print ('g_dis_loss', g_dis_loss)
            alpha = 0.05
            g_loss = torch.sum(g_dis_loss + alpha * g_gen_loss)
            print('g_loss', g_loss)
            g_cost_avg += g_loss.data[0]
            g_loss.backward()
            g_optim.step()
#             info = {
#                   'g_loss' : g_loss.data[0],
#                   'fake_score_mean' : fake_score,
#             }
#             for tag,value in info.items():
#                 logger.scalar_summary(tag, value, counter,i)
        
        
        if n_updates == 1:
            print("Epoch [%d/%d], step[%d/%d], d_loss: %.4f, D(x): %.2f, time: %4.4f"
                  %(current_epoch, num_epoch, i+1, num_batch, d_loss.data[0],  real_score, time.time()-start_time) )
        else:
            print("Epoch [%d/%d], step[%d/%d], d_loss: %.4f, g_loss: %.4f, D(x): %.2f, D(G(x)): %.2f, time: %4.4f"
                  %(current_epoch, num_epoch, i+1, num_batch, d_loss.data[0], g_loss.data[0],
                    real_score, fake_score, time.time()-start_time) )
        counter += 1
        n_updates += 1
    d_cost_avg /= num_batch
    g_cost_avg /= num_batch
    print('Epoch:', current_epoch, ' train_loss->', (d_cost_avg, g_cost_avg))
    torch.save(generator.state_dict(), r'C:\Users\pasan\Documents\Notebooks\Saliency\SavedStates/generator.pkl')
    torch.save(discriminator.state_dict(), r'C:\Users\pasan\Documents\Notebooks\Saliency\SavedStates/discriminator.pkl')
    predict(generator, validation_sample, current_epoch, DIR_TO_SAVE)
torch.save(generator.state_dict(),r'C:\Users\pasan\Documents\Notebooks\Saliency\SavedStates/generator.pkl')
torch.save(discriminator.state_dict(), r'C:\Users\pasan\Documents\Notebooks\Saliency\SavedStates/discriminator.pkl')
print('Done')


            
    

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

Training Discriminator...
before convs1
after convs1 torch.Size([20, 3, 192, 256])
after convs2 torch.Size([20, 32, 96, 128])
after convs3 torch.Size([20, 64, 48, 64])
after convs4 torch.Size([20, 64, 48, 64])
after convs5 torch.Size([20, 64, 24, 32])
x size torch.Size([20, 64, 24, 32])
torch.Size([64, 24, 32])
torch.Size([20, 49152])


C:\Users\pasan\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


D_real_loss =  tensor(0.7687)
OUTPUTS tensor([0.4637, 0.4636, 0.4637, 0.4637, 0.4637, 0.4636, 0.4637, 0.4635, 0.4636,
        0.4637, 0.4636, 0.4636, 0.4637, 0.4637, 0.4636, 0.4636, 0.4636, 0.4635,
        0.4636, 0.4636], grad_fn=<SqueezeBackward0>)
D_LOSS tensor(-15.3731, grad_fn=<SumBackward0>)


C:\Users\pasan\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
C:\Users\pasan\Anaconda3\lib\site-packages\ipykernel_launcher.py:79: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


...

Training Discriminator...
before convs1
after convs1 torch.Size([20, 3, 192, 256])
after convs2 torch.Size([20, 32, 96, 128])
after convs3 torch.Size([20, 64, 48, 64])
after convs4 torch.Size([20, 64, 48, 64])
after convs5 torch.Size([20, 64, 24, 32])
x size torch.Size([20, 64, 24, 32])
torch.Size([64, 24, 32])
torch.Size([20, 49152])
D_real_loss =  tensor(1.1728)
OUTPUTS tensor([0.3103, 0.3089, 0.3093, 0.3110, 0.3093, 0.3079, 0.3096, 0.3090, 0.3092,
        0.3089, 0.3092, 0.3106, 0.3100, 0.3086, 0.3091, 0.3098, 0.3105, 0.3095,
        0.3091, 0.3100], grad_fn=<SqueezeBackward0>)
D_LOSS tensor(-23.4568, grad_fn=<SumBackward0>)
Epoch [2/3], step[1/500], d_loss: -23.4568, D(x): 0.31, time: 683.7638
Training Generator...
before convs1
after convs1 torch.Size([20, 3, 192, 256])
after convs2 torch.Size([20, 32, 96, 128])
after convs3 torch.Size([20, 64, 48, 64])
after convs4 torch.Size([20, 64, 48, 64])
after convs5 torch.Size([20, 64, 24, 32])
x size torch.Size([20, 64, 24, 32])
torch.Siz

 67%|███████████████████████████████████████████████████████▎                           | 2/3 [20:31<10:39, 639.71s/it]

Training Discriminator...
before convs1
after convs1 torch.Size([20, 3, 192, 256])
after convs2 torch.Size([20, 32, 96, 128])
after convs3 torch.Size([20, 64, 48, 64])
after convs4 torch.Size([20, 64, 48, 64])
after convs5 torch.Size([20, 64, 24, 32])
x size torch.Size([20, 64, 24, 32])
torch.Size([64, 24, 32])
torch.Size([20, 49152])
D_real_loss =  tensor(1.2741)
OUTPUTS tensor([0.2790, 0.2791, 0.2808, 0.2794, 0.2801, 0.2798, 0.2802, 0.2789, 0.2784,
        0.2791, 0.2801, 0.2803, 0.2796, 0.2795, 0.2796, 0.2795, 0.2801, 0.2803,
        0.2800, 0.2796], grad_fn=<SqueezeBackward0>)
D_LOSS tensor(-25.4829, grad_fn=<SumBackward0>)
Epoch [3/3], step[1/500], d_loss: -25.4829, D(x): 0.28, time: 1239.4224
Training Generator...
before convs1
after convs1 torch.Size([20, 3, 192, 256])
after convs2 torch.Size([20, 32, 96, 128])
after convs3 torch.Size([20, 64, 48, 64])
after convs4 torch.Size([20, 64, 48, 64])
after convs5 torch.Size([20, 64, 24, 32])
x size torch.Size([20, 64, 24, 32])
torch.Si

100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [31:19<00:00, 642.31s/it]


Done
